In [1]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import json
import re
import rapidfuzz

In [2]:
newoag=pd.read_json('oag_qa_20230512.json',lines=True)

In [3]:
newoag_title=newoag[['title','answers']]

In [4]:
newoag_title_dict=newoag_title.to_dict()

In [5]:
import os
import glob

folder_path = "/Users/judyzhu/Desktop/2023 summer/oagqa-topic-v2"  # Replace with the actual folder path
files = glob.glob(os.path.join(folder_path, "*train*"))
merged_data=[]
for file in files: 
    with open(file, 'r') as file:
        json_data = json.load(file)
        df=pd.DataFrame(json_data)
        merged_data.append(df)
merged_data
joined_data=pd.concat(merged_data)

In [6]:
tsvfiles = glob.glob(os.path.join(folder_path, "*questions.tsv*"))
#data=pandas.read_csv('filename.tsv',sep='\t')
merged_data=[]
for file in tsvfiles: 
    df=pd.read_csv(file,sep='\t',names=['question','answer'])
    merged_data.append(df)
joined_data=pd.concat(merged_data)

In [ ]:

# def match_names(name, list_names, min_score=0):
#     matching_names = []
#     for x in list_names:
#         score = fuzz.ratio(name, x)
#         if score > 90:  # Check if the score is greater than 90
#             matching_names.append((x, score))
#     return matching_names

In [30]:
newoag.head()

,_id,qid,score,url,title,body,tags,answers
0,{'$oid': '645a083eeae567d13a86162e'},4834,6,https://patents.stackexchange.com/questions/48...,Cloud Computing - Suggesting customers migrate...,<p><strong>AN OVERBROAD PATENT ON suggesting c...,"[prior-art-request, pre-grant, ibm, cloud-stor...","[{'aid': '4956', 'qid': '4834', 'is_accepted':..."
1,{'$oid': '645a0840eae567d13a86162f'},13157,6,https://patents.stackexchange.com/questions/13...,Requesting prior art on Google machine learnin...,<p>Google is attempting to patent well known c...,"[prior-art-request, google, wo2014105866a1, us...","[{'aid': '13206', 'qid': '13157', 'is_accepted..."
2,{'$oid': '645a0842eae567d13a861630'},12028,4,https://patents.stackexchange.com/questions/12...,EDA software (IC design),<p>Looking for prior art on this:</p>\n\n<p><a...,"[prior-art-request, pre-grant, us20130290834, ...","[{'aid': '21228', 'qid': '12028', 'is_accepted..."
3,{'$oid': '645a0842c73f28c9898f4260'},110634,84,https://softwareengineering.stackexchange.com/...,Why would it ever be possible for Java to be f...,<p>Sometimes Java outperforms C++ in benchmark...,"[java, c++, performance]","[{'aid': '430340', 'qid': '110634', 'is_accept..."
4,{'$oid': '645a0842c73f28c9898f4261'},322714,76,https://softwareengineering.stackexchange.com/...,Are stacks the only reasonable way to structur...,<p>Most architectures I've seen rely on a call...,[computer-architecture],"[{'aid': '322732', 'qid': '322714', 'is_accept..."


In [7]:
# joined_data_name=joined_data[['question','answer']]
# joined_data_name

,question,answer
0,Example of a quantum-mechanical theory with no...,"['toeplitz quantization of k\\""ahler manifolds..."
1,How can transverse waves on a string carry lon...,['The missing wave momentum mystery']
2,How do contact transformations differ from can...,['contact hamiltonian mechanics']
3,Reference for a type of &quot;multi-hamiltonia...,['Classical and quantum Nambu mechanics']
4,Issues with Newton&#39;s third law and Euler&#...,"[""mansuripur's paradox""]"
...,...,...
77,What do blind people dream when they dream?,['Visual imagery without visual perception?']
78,What is the cognitive neural mechanism of atte...,['Attention Induced Gain Stabilization in Broa...
79,"How to evaluate the study of ""only 39 of the 1...",['Statistics: <i>P</i> values are just the tip...
80,What do you think of the findings published in...,['Human Hippocampal Neurogenesis Persists thro...


In [17]:
newoag_title=newoag.set_index('title')
newoag_title.head(3)

,_id,qid,score,url,body,tags,answers
title,,,,,,,
Cloud Computing - Suggesting customers migrate to the cloud (IBM) - Patent Application - PRIOR ART REQUEST,{'$oid': '645a083eeae567d13a86162e'},4834,6,https://patents.stackexchange.com/questions/48...,<p><strong>AN OVERBROAD PATENT ON suggesting c...,"[prior-art-request, pre-grant, ibm, cloud-stor...","[{'aid': '4956', 'qid': '4834', 'is_accepted':..."
Requesting prior art on Google machine learning patent (US 2014/0180986),{'$oid': '645a0840eae567d13a86162f'},13157,6,https://patents.stackexchange.com/questions/13...,<p>Google is attempting to patent well known c...,"[prior-art-request, google, wo2014105866a1, us...","[{'aid': '13206', 'qid': '13157', 'is_accepted..."
EDA software (IC design),{'$oid': '645a0842eae567d13a861630'},12028,4,https://patents.stackexchange.com/questions/12...,<p>Looking for prior art on this:</p>\n\n<p><a...,"[prior-art-request, pre-grant, us20130290834, ...","[{'aid': '21228', 'qid': '12028', 'is_accepted..."


In [18]:
newoag_title['old']=False

In [27]:
# joined_data.loc[1,'question']

1    How can transverse waves on a string carry lon...
1     Does training affect the tactile oblique effect?
1    Is density functional theory an ab initio method?
1                         Vacuum and repulsive gravity
1             Clustering a noisy data or with outliers
                           ...                        
1    Relation between left and right coset represen...
1    What is the largest possible mass for a stable...
1               Dark matter and dark energy references
1    Learning capacity: Deep Learning vs Traditiona...
1    Do neurons have a special division of labor or...
Name: question, Length: 87, dtype: object

In [44]:
#find only the one that matches
def closest(title, old):
    for i in range(len(old)):
        old['score']=rapidfuzz.fuzz.ratio(title, old['question'].iloc[i])
    return old['score'].max()
        
        

In [51]:
def find_non_matching_titles(new, old, min_score=90):
    for i in range(len(new)):
        title=new['title'].iloc[i]
        #if rapidfuzz.fuzz.ratio(closest(title, old[['question']]))<min_score:
        if closest(title, old)<min_score:
            new['old'].iloc[i]=True
    return new
            


In [52]:
closest('Cloud Computing - Suggesting customers migrate to the cloud (IBM) - Patent Application - PRIOR ART REQUEST', joined_data)

33.513513513513516

In [ ]:
find_non_matching_titles(newoag, joined_data, min_score=90)

/Users/judyzhu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
# joined_data_namedict=joined_data['question'].to_dict()

In [9]:
# oldquestion=list(joined_data['question'])
# newquestion=list(newoag['title'])

In [ ]:
# from fuzzywuzzy import fuzz, process
# import pandas as pd

# def find_non_matching_titles(titles, questions, min_score=90):
#     non_matching_titles = []
#     commonthreshold=0
#     questions_words = get_unique_words(questions)
#     for title in titles:
#         matched = False
#         title_words = set(title.lower().split())
#         for question in questions:
#             if len(title_words.intersection(questions_words))==0:
#                 non_matching_titles.append(title)
#             else: 
#                 score = rapidfuzz.fuzz.ratio(title, question)
#                 if score > min_score:
#                     matched = True
#                     break
#                 if not matched:
#                     non_matching_titles.append(title)
#     return non_matching_titles
